# 1.) Import the Credit Card Fraud Data From CCLE

In [ ]:
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
drive.mount('/content/gdrive/', force_remount = True)

Mounted at /content/gdrive/


In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/ECON441B/Week3/fraudTest.csv")

In [ ]:
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


# 2.) Select four columns to use as features (one just be trans_date_trans)

In [ ]:
df.loc[df["gender"]=="M","gender"] = 1
df.loc[df["gender"]=="F","gender"] = 0

# 3.) Create a your own variable out of trans_date. Create dummies for factor vars

In [ ]:
type(df["trans_date_trans_time"][0])

str

In [ ]:
df["trans_date_trans_time"] = pd.to_datetime(df["trans_date_trans_time"])

In [ ]:
df["time_var"] = [i.hour for i in df["trans_date_trans_time"]]

In [ ]:
df_select = df[["time_var", "category", "amt", "gender","is_fraud"]]

In [ ]:
X = pd.get_dummies(df_select,columns=["category"]).drop(["is_fraud"],axis =1)
y = df["is_fraud"]

In [ ]:
X.head()

,time_var,amt,gender,category_entertainment,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel
0,12,2.86,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,12,29.84,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,12,41.28,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,12,60.05,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,12,3.19,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


# 5.) Train a Logistic regression.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression().fit(X_normalized, y)

# 6.) The company you are working for wants to target at a False Positive rate of 5% what threshold should you use? (Use oversampled data)

In [ ]:
# ASK chatgpt
# If you dont like the code ask in a different way
from sklearn.metrics import confusion_matrix

In [ ]:
# predict class probabilities
probs = model.predict_proba(X_normalized)

In [ ]:
# keep only the positive class
pos_probs = probs[:,1]

# set threshold for false positive rate
fp_rate = 0.05
threshold = sorted(pos_probs)[int((1-fp_rate)*len(pos_probs))]

In [ ]:
# make predictions
predictions = (pos_probs >= threshold).astype(int)

# calculate confusion matrix
confusion_matrix(y, predictions)

array([[526763,  26811],
       [  1170,    975]])

# 7.) If the company makes .02*amt on True transactions and loses -amt on False (Use original data)

In [ ]:
df_temp = df_select.copy()

In [ ]:
df_temp["pred"] = model.predict(X_normalized)


In [ ]:
df_temp = df_temp[["pred", "is_fraud", "amt"]]

In [ ]:
df_temp.head()
# Which combinations of "pred" and "if_fraud" do we profit .02*amt, which cases do 
# we lose -amt??

,pred,is_fraud,amt
0,0,0,2.86
1,0,0,29.84
2,0,0,41.28
3,0,0,60.05
4,0,0,3.19


In [ ]:
# The answer should be one single numb
profit=0
for i in range(len(df_temp["amt"])):
  if df_temp.pred[i]==0:
      if df_temp.is_fraud[i]==0:
        profit = profit + df_temp.amt[i]*0.02
      else:
        profit = profit - df_temp.amt[i]
  else:
    continue
print(profit)

-400419.842000019


# 8.) Using Logistic Regression Lasso to inform you. Would you use the selected features in a trusted prediction model?

In [ ]:
model2 = LogisticRegression(penalty = "l1",solver = 'liblinear').fit(X_normalized, y)

In [ ]:
# If most or all your variables go to 0 => Your data is garbage
# The regularization will tell us if our model has significance
# This of using coefficient strength similar to r^2

In [ ]:
model2.coef_.ravel()

array([ 0.8446838 ,  0.31393331,  0.06246354, -0.18521863, -0.20340596,
        0.38162439,  0.18795232,  0.67744267, -0.21378449, -0.28524557,
       -0.2619239 ,  0.40599839,  0.        , -0.15829062,  0.35337865,
        0.04761781, -0.90046158])

From coefficients, not all coefficients go to zero, so some features are still useful, but we can say gender feature is garbage.